In [14]:
!pip3 install sentencepiece tqdm

In [7]:
!pip3 install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 143.3 MB/s eta 0:00:00


In [ ]:
!pip3 install torch

In [ ]:
!pip3 install transformers pandas numpy

In [18]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import datetime

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-uk-ru")

model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-uk-ru").to('cuda')
# model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-uk-ru")


def trans(text):
        inputs = tokenizer(text, return_tensors="pt", truncation=True)
        translation_ids = model.generate(**inputs)
        translation_text = tokenizer.batch_decode(translation_ids, skip_special_tokens=True)[0]
        print(datetime.datetime.now())
        return translation_text

def split_string_into_chunks(input_string, chunk_size):
    return [input_string[i:i+chunk_size + 25] for i in range(0, len(input_string), chunk_size)]


def read_file(file_path, chunk_size: int = 1500):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    return split_string_into_chunks(content, chunk_size)


def translate_file(input, output):
        a = read_file(input)
        # tr = [trans(ch) for ch in a]
        tr = [translate(ch) for ch in a]
        with open(output, 'w') as file:
            for line in tr:
                file.write(line + '\n')


# translate_file(args[0], args[1])

In [9]:
def translate(text):
    input_ids = tokenizer(text, return_tensors="pt").input_ids.to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(input_ids=input_ids)
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def translate_batch(texts):
    input_ids = tokenizer(texts, return_tensors="pt", padding=True, truncation=True).to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(**input_ids)
    
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [12]:
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

In [17]:
import pandas as pd

In [16]:
bs = 16
from tqdm import tqdm
import df 
for chunk in tqdm(
    batch(df['tgt_abs'].tolist(), bs),
    total=len(df['tgt_abs']) // bs
):
    translate_batch(chunk)
    torch.cuda.empty_cache()

NameError: name 'df' is not defined

In [19]:
translate_file('aref.doc.txt.head', 'aref.doc.txt.head.tr')